**Requires**

**Output**

## Imports

In [18]:
import scanpy as sc
import matplotlib.pyplot as plt
import numpy as np 
import pandas as pd
import os

from chemCPA.paths import DATA_DIR, PROJECT_DIR, ROOT

sc.set_figure_params(dpi=80, frameon=False)
sc.logging.print_header()

scanpy==1.9.3 anndata==0.8.0 umap==0.5.3 numpy==1.21.6 scipy==1.7.3 pandas==1.3.5 scikit-learn==1.0.2 statsmodels==0.13.2 pynndescent==0.5.7


In [19]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


### Loading LINCS and reference data

In [20]:

# print(PROJECT_DIR)
# # print the directory of home from the root directory
# # Root directory: /home/ubuntu/miniconda3/envs/chemical_CPA/lib/python3.7/site-packages
# file_path = os.path.abspath('lincs_full.h5ad')
# print(f"Absolute path of lincs_full_pp.h5ad: {file_path}")
# !ls -l $DATA_DIR   


In [21]:
import os
from pathlib import Path

full = False
load_adata = True 

DATA_DIR = Path('/storage/kasperek/sc-transformer/chemCPA_fork/datasets/')

if full:
    adata_in = DATA_DIR / 'lincs_full_pp.h5ad'
    adata_out = DATA_DIR / 'lincs_full_smiles.h5ad'
else:
    adata_in = DATA_DIR / 'lincs_pp.h5ad'
    adata_out = DATA_DIR / 'lincs_smiles.h5ad'

print(f'Input: {adata_in}')
    
adata = sc.read(adata_in) if load_adata else None
adata.obs.head()

Input: /storage/kasperek/sc-transformer/chemCPA_fork/datasets/lincs_pp.h5ad


,cell_id,det_plate,det_well,lincs_phase,pert_dose,pert_dose_unit,pert_id,pert_iname,pert_mfc_id,pert_time,...,rna_well,batch,condition,cell_type,dose_val,cov_drug_dose_name,control,split,cov_drug_name,eval_category
REP.A001_A375_24H_X1_B22:B13-2-0,A375,REP.A001_A375_24H_X1_B22,B13,2,10.000000,um,BRD-A25234499,aminoglutethimide,BRD-A25234499-001-18-3,24.0,...,nan,0,aminoglutethimide,A375,10.000000,A375_aminoglutethimide_10.0,0,train,A375_aminoglutethimide,A375_aminoglutethimide
REP.A001_A375_24H_X1_B22:B14-2-0,A375,REP.A001_A375_24H_X1_B22,B14,2,3.333330,um,BRD-A25234499,aminoglutethimide,BRD-A25234499-001-18-3,24.0,...,nan,0,aminoglutethimide,A375,3.333330,A375_aminoglutethimide_3.33333,0,train,A375_aminoglutethimide,A375_aminoglutethimide
REP.A001_A375_24H_X1_B22:B15-2-0,A375,REP.A001_A375_24H_X1_B22,B15,2,1.111110,um,BRD-A25234499,aminoglutethimide,BRD-A25234499-001-18-3,24.0,...,nan,0,aminoglutethimide,A375,1.111110,A375_aminoglutethimide_1.11111,0,train,A375_aminoglutethimide,A375_aminoglutethimide
REP.A001_A375_24H_X1_B22:B16-2-0,A375,REP.A001_A375_24H_X1_B22,B16,2,0.370370,um,BRD-A25234499,aminoglutethimide,BRD-A25234499-001-18-3,24.0,...,nan,0,aminoglutethimide,A375,0.370370,A375_aminoglutethimide_0.37037,0,train,A375_aminoglutethimide,A375_aminoglutethimide
REP.A001_A375_24H_X1_B22:B17-2-0,A375,REP.A001_A375_24H_X1_B22,B17,2,0.123457,um,BRD-A25234499,aminoglutethimide,BRD-A25234499-001-18-3,24.0,...,nan,0,aminoglutethimide,A375,0.123457,A375_aminoglutethimide_0.123457,0,train,A375_aminoglutethimide,A375_aminoglutethimide


Checking number of drugs for LINCS

In [22]:
pert_id_unique = pd.Series(np.unique(adata.obs.pert_id))
print(f"# of unique perturbations: {len(pert_id_unique)}")

# of unique perturbations: 1120


Loading reference dataframe that contains SMILES 
restricting to `'pert_id'` and `'canonical_smiles'`

In [23]:
reference_df = pd.read_csv(DATA_DIR /'GSE92742_Broad_LINCS_pert_info.txt', delimiter = "\t")
reference_df = reference_df.loc[reference_df.pert_id.isin(pert_id_unique), ['pert_id', 'canonical_smiles']]
reference_df.canonical_smiles.value_counts()

-666                                                                                                6
restricted                                                                                          2
CS(=O)(=O)CCNCc1ccc(o1)-c1ccc2ncnc(Nc3ccc(OCc4cccc(F)c4)c(Cl)c3)c2c1                                2
Oc1ccc(cc1)N1CCN(CC1)[S+]([O-])(=O)c1ccc2NC(=O)Cc2c1                                                1
Cc1c(CCOC(=O)c2ccc(N3CCOCC3)c(c2)[N+](=O)[O-])sc[n+]1CC#N                                           1
                                                                                                   ..
CNC(=O)c1cc(Oc2ccc(NC(=O)Nc3ccc(Cl)c(c3)C(F)(F)F)cc2)ccn1                                           1
COc1ccc(cc1)[C@@H]1Sc2ccccc2N(CCN(C)C)C(=O)[C@@H]1OC(C)=O                                           1
C[C@H](CO)N1C[C@@H](C)[C@@H](CN(C)C(=O)CCC(F)(F)F)OCCCC[C@H](C)Oc2ccc(NS(=O)(=O)c3ccccc3)cc2C1=O    1
COc1ccc(cc1)[Si](C)(C)[C@H]2[C@H](C)[C@]3(O[C@@H]2CCO)C(=O)N(Cc4cccc(c4)N5CCOC5=O)

In [24]:
cond = ~pert_id_unique.isin(reference_df.pert_id)
print(f"From {len(pert_id_unique)} total drugs, {cond.sum()} were not part of the reference dataframe.")

From 1120 total drugs, 132 were not part of the reference dataframe.


Adding `'canoncical_smiles'` column to `adata.obs` via `pd.merge`

In [25]:
adata.obs = adata.obs.reset_index().merge(reference_df, how="left").set_index('index')

Removing invalid SMILES strings 

In [26]:
adata.obs.pert_id

index
REP.A001_A375_24H_X1_B22:B13-2-0    BRD-A25234499
REP.A001_A375_24H_X1_B22:B14-2-0    BRD-A25234499
REP.A001_A375_24H_X1_B22:B15-2-0    BRD-A25234499
REP.A001_A375_24H_X1_B22:B16-2-0    BRD-A25234499
REP.A001_A375_24H_X1_B22:B17-2-0    BRD-A25234499
                                        ...      
PCLB003_PC3_24H_X3_B13:A04-1-1               DMSO
PCLB003_PC3_24H_X3_B13:A05-1-1               DMSO
PCLB003_PC3_24H_X3_B13:A06-1-1               DMSO
PCLB003_PC3_24H_X3_B13:B04-1-1               DMSO
PCLB003_PC3_24H_X3_B13:B05-1-1               DMSO
Name: pert_id, Length: 199620, dtype: object

In [27]:
reference_df

,pert_id,canonical_smiles
102,BRD-A00474148,Oc1ccc(cc1)N1CCN(CC1)[S+]([O-])(=O)c1ccc2NC(=O...
108,BRD-A00827783,Cn1c2ncn(CC(O)CO)c2c(=O)n(C)c1=O
119,BRD-A01787639,COc1ccccc1N1CCN(CC(O)COc2cccc3ccccc23)CC1
131,BRD-A02367930,C[C@]12CCC3C(CCc4cc(O)ccc34)C1CC[C@@]2(O)C#C
155,BRD-A03946276,[O-][N+](=O)c1ccc2NC(=O)C3(NNc4ccccc4O3)c2c1
...,...,...
20470,BRD-U37049823,restricted
20490,BRD-U85918428,-666
20498,BRD-U97083655,restricted
26389,CMAP-PRISM-TP7,-666


In [28]:
adata.obs.loc[:, 'canonical_smiles'] = adata.obs.canonical_smiles.astype('str')
invalid_smiles = adata.obs.canonical_smiles.isin(['-666', 'restricted', 'nan'])
print(f'Among {len(adata)} observations, {100*invalid_smiles.sum()/len(adata):.2f}% ({invalid_smiles.sum()}) have an invalid SMILES string')
adata = adata[~invalid_smiles]

Among 199620 observations, 11.54% (23033) have an invalid SMILES string


Remove invalid `'pert_dose'` value: `-666`

In [29]:
cond = adata.obs.pert_dose.isin([-666])
adata = adata[~cond]
print(f"A total of {cond.sum()} observations have invalid dose values")

A total of 0 observations have invalid dose values


In [30]:
drugs_validation = adata.obs.canonical_smiles.value_counts() < 6
valid_drugs = drugs_validation.index[~drugs_validation]
cond = adata.obs.canonical_smiles.isin(valid_drugs)
print(f"A total of {(~cond).sum()} observation belong to drugs which do not have enough replicates")
adata = adata[cond]

A total of 0 observation belong to drugs which do not have enough replicates


Checking that SMILES are valid according to `rdkit` 

In [31]:
from rdkit import Chem

def check_smiles(smiles):
    m = Chem.MolFromSmiles(smiles,sanitize=False)
    if m is None:
        print('invalid SMILES')
        return False
    else:
        try:
            Chem.SanitizeMol(m)
        except:
            print('invalid chemistry')
            return False
    return True

def remove_invalid_smiles(dataframe, smiles_key: str = 'SMILES', return_condition: bool = False):
    unique_drugs = pd.Series(np.unique(dataframe[smiles_key]))
    valid_drugs = unique_drugs.apply(check_smiles)
    print(f"A total of {(~valid_drugs).sum()} have invalid SMILES strings")
    _validation_map = dict(zip(unique_drugs, valid_drugs))
    cond = dataframe[smiles_key].apply(lambda x: _validation_map[x])
    if return_condition: 
        return cond
    dataframe = dataframe[cond].copy()
    return dataframe

adata

View of AnnData object with n_obs × n_vars = 176587 × 978
    obs: 'cell_id', 'det_plate', 'det_well', 'lincs_phase', 'pert_dose', 'pert_dose_unit', 'pert_id', 'pert_iname', 'pert_mfc_id', 'pert_time', 'pert_time_unit', 'pert_type', 'rna_plate', 'rna_well', 'batch', 'condition', 'cell_type', 'dose_val', 'cov_drug_dose_name', 'control', 'split', 'cov_drug_name', 'eval_category', 'canonical_smiles'
    var: 'pr_gene_title', 'pr_is_lm', 'pr_is_bing'
    uns: 'rank_genes_groups_cov'

In [32]:
cond = remove_invalid_smiles(adata.obs, smiles_key='canonical_smiles', return_condition=True)
adata = adata[cond]

A total of 0 have invalid SMILES strings


### Add additional drugbank info to `adata.obs`

In [33]:
from os.path import exists

drugbank_path = DATA_DIR / 'drug_bank'/ 'drugbank_all.csv'
if exists(drugbank_path): 
    drugbank_df = pd.read_csv(drugbank_path)
else: 
    print(f'Invalid path: {drugbank_path}')

Invalid path: /storage/kasperek/sc-transformer/chemCPA_fork/datasets/drug_bank/drugbank_all.csv


In [34]:
from rdkit.Chem import CanonSmiles

drugs_canonical = pd.Series(np.unique(adata.obs.canonical_smiles)).apply(CanonSmiles)
db_canonical_smiles = drugbank_df.SMILES.apply(CanonSmiles)
n_overlap = drugs_canonical.isin(db_canonical_smiles).sum()
print(f'From a total of {len(drugs_canonical)}, {100*n_overlap/len(drugs_canonical):.2f}% ({n_overlap}) is also available in drugbank.')

NameError: name 'drugbank_df' is not defined

In [ ]:
cond = db_canonical_smiles.isin(drugs_canonical)
drugbank_df.loc[cond, ['ATC_level_1']].value_counts()

ATC_level_1                               
an                                            393
NERVOUS SYSTEM                                184
CARDIOVASCULAR SYSTEM                         152
ANTINEOPLASTIC AND IMMUNOMODULATING AGENTS     98
ALIMENTARY TRACT AND METABOLISM                93
ANTIINFECTIVES FOR SYSTEMIC USE                81
RESPIRATORY SYSTEM                             78
GENITO URINARY SYSTEM AND SEX HORMONES         60
MUSCULO-SKELETAL SYSTEM                        58
DERMATOLOGICALS                                51
SENSORY ORGANS                                 47
INSECTICIDES AND REPELLENTS                    29
VARIOUS                                        21
BLOOD AND BLOOD FORMING ORGANS                 17
EXCL. SEX HORMONES AND INSULINS                 7
dtype: int64

### Add `train`, `test`, `ood` split for full lincs dataset (if not already part in `adata.obs`)

In [ ]:
from sklearn.model_selection import train_test_split

if 'split' not in list(adata.obs):
    print("Addig 'split' to 'adata.obs'.")
    unique_drugs = np.unique(adata.obs.canonical_smiles)
    drugs_train, drugs_tmp = train_test_split(unique_drugs, test_size=0.2)
    drugs_val, drugs_test = train_test_split(drugs_tmp, test_size=0.5)

    adata.obs['split'] = 'train'
    adata.obs.loc[adata.obs.canonical_smiles.isin(drugs_val), 'split'] = 'test'
    adata.obs.loc[adata.obs.canonical_smiles.isin(drugs_test), 'split'] = 'ood'

### Check that `.obs.split=='test'` has sufficient samples for `pert_id` and `cell_id`

In [ ]:
adata.obs.split.value_counts()

train    699463
test     133383
ood        7831
Name: split, dtype: int64

In [ ]:
cond_test = adata.obs.split.isin(['test'])
adata.obs.loc[cond_test, 'cell_id'].value_counts()

MCF7        18766
VCAP        17841
PC3         17414
A375        11188
HT29        10970
            ...  
HUES3          93
SNUC5          89
NCIH1694       86
SKMEL28        77
HS27A           9
Name: cell_id, Length: 82, dtype: int64

In [ ]:
adata.obs.loc[cond_test, 'pert_id'].value_counts()

BRD-K60230970    904
BRD-K50691590    882
BRD-K81418486    694
DMSO             618
BRD-A19500257    538
                ... 
BRD-K79746073      1
BRD-K01751906      1
BRD-K10558099      1
BRD-K29360016      1
BRD-K23208702      1
Name: pert_id, Length: 15649, dtype: int64

In [ ]:
pert_count_treshold = 5
cov_count_treshold = 20

pert_id_neg = adata.obs.loc[cond_test, 'pert_id'].value_counts() < pert_count_treshold
print(f"pert_id: {pert_id_neg.sum()}/{len(pert_id_neg)} converted back to 'train' due to insufficient # of samples.")

cov_id_neg = adata.obs.loc[cond_test, 'cell_id'].value_counts() < cov_count_treshold
print(f"cell_id: {cov_id_neg.sum()}/{len(cov_id_neg)} converted back to 'train' due to insufficient # of samples.")

cond = cond_test & adata.obs.pert_id.isin(pert_id_neg.index[pert_id_neg])
cond |= cond_test & adata.obs.cell_id.isin(cov_id_neg.index[cov_id_neg])

pert_id: 9257/15649 converted back to 'train' due to insufficient # of samples.
cell_id: 1/82 converted back to 'train' due to insufficient # of samples.


In [ ]:
adata.obs['split1'] = adata.obs.split.copy()
adata.obs.loc[cond, 'split1'] = 'train'
print(f"split['test']: {cond.sum()}/{len(cond)} samples are converted back to 'train'.")

split['test']: 18885/840677 samples are converted back to 'train'.


In [ ]:
adata.obs.split1.value_counts()

train    718348
test     114498
ood        7831
Name: split1, dtype: int64

### Add random split

In [ ]:
adata.obs_names

Index(['REP.A001_A375_24H_X1_B22:B13-2', 'REP.A001_A375_24H_X1_B22:B14-2',
       'REP.A001_A375_24H_X1_B22:B15-2', 'REP.A001_A375_24H_X1_B22:B16-2',
       'REP.A001_A375_24H_X1_B22:B17-2', 'REP.A001_A375_24H_X1_B22:B18-2',
       'REP.A001_A375_24H_X1_B22:B19-2', 'REP.A001_A375_24H_X1_B22:B20-2',
       'REP.A001_A375_24H_X1_B22:B21-2', 'REP.A001_A375_24H_X1_B22:B22-2',
       ...
       'PCLB003_PC3_24H_X3_B13:P15-1', 'PCLB003_PC3_24H_X3_B13:P16-1',
       'PCLB003_PC3_24H_X3_B13:P17-1', 'PCLB003_PC3_24H_X3_B13:P18-1',
       'PCLB003_PC3_24H_X3_B13:P19-1', 'PCLB003_PC3_24H_X3_B13:P20-1',
       'PCLB003_PC3_24H_X3_B13:P21-1', 'PCLB003_PC3_24H_X3_B13:P22-1',
       'PCLB003_PC3_24H_X3_B13:P23-1', 'PCLB003_PC3_24H_X3_B13:P24-1'],
      dtype='object', name='index', length=840677)

In [ ]:
train_obs, val_test_obs = train_test_split(adata.obs_names, test_size=0.15)
val_obs, test_obs = train_test_split(val_test_obs, test_size=0.5)

In [ ]:
adata.obs['random_split'] = ''
adata.obs.loc[train_obs, 'random_split'] = 'train'
adata.obs.loc[val_obs, 'random_split'] = 'test'
adata.obs.loc[test_obs, 'random_split'] = 'ood'


adata.obs['random_split'].value_counts() 

train    714575
test      63051
ood       63051
Name: random_split, dtype: int64

Check that perturbations occur in train split (no explicit ood!)

In [ ]:
len(adata.obs.loc[adata.obs.random_split == 'train', 'pert_id'].unique()) 

17775

In [ ]:
len(adata.obs.pert_id.unique())

17775

## Safe adata

In [ ]:
adata.write(adata_out)
adata

... storing 'pert_id' as categorical
... storing 'canonical_smiles' as categorical
... storing 'random_split' as categorical


AnnData object with n_obs × n_vars = 840677 × 978
    obs: 'cell_id', 'det_plate', 'det_well', 'lincs_phase', 'pert_dose', 'pert_dose_unit', 'pert_id', 'pert_iname', 'pert_mfc_id', 'pert_time', 'pert_time_unit', 'pert_type', 'rna_plate', 'rna_well', 'condition', 'cell_type', 'dose_val', 'cov_drug_dose_name', 'control', 'split', 'canonical_smiles', 'split1', 'random_split'
    var: 'pr_gene_title', 'pr_is_lm', 'pr_is_bing'
    uns: 'cydata_pull', 'rank_genes_groups_cov'

In [ ]:
# adata.obs['drug_dose_name']

### Loading the result for `adata_out`

In [ ]:
adata = sc.read(adata_out)

**Additional**: Check that `adata.uns[rank_genes_groups_cov]` has all entries in `adata.obs.cov_drug_dose_name` as keys

In [ ]:
for i, k in enumerate(adata.obs.cov_drug_dose_name.unique()):
    try: 
        adata.uns['rank_genes_groups_cov'][k]
    except: 
        print(f"{i}: {k}") if 'DMSO' not in k else None